# Preprocess

In [ ]:
!pip install niapy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 176 kB 4.8 MB/s 


In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import gdown
from sklearn.model_selection import train_test_split, cross_val_score

In [ ]:
!gdown --id 1OLnlno2zp6HXmIxNdNcK6l3hidH4ttVr
!gdown --id 1cqgF0TDS6-wZql4NayQvPNvJx9SBf_CD

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1OLnlno2zp6HXmIxNdNcK6l3hidH4ttVr
To: /content/boston_train.xlsx
100% 34.4k/34.4k [00:00<00:00, 49.9MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1cqgF0TDS6-wZql4NayQvPNvJx9SBf_CD
To: /content/boston_test.xlsx
100% 13.1k/13.1k [00:00<00:00, 21.7MB/s]


In [ ]:
train = pd.read_excel('boston_train.xlsx')
test = pd.read_excel('boston_test.xlsx')

In [ ]:
train.isnull().sum()

CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
PRICE      0
dtype: int64

In [ ]:
scaler = StandardScaler()

In [ ]:
X = train[['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']]
y = train[['PRICE']]

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=1234)

In [ ]:
X_test = test[['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']]
y_test = test[['PRICE']]

In [ ]:
X_train = scaler.fit_transform(X_train)
X_val = scaler.fit_transform(X_val)
X_test = scaler.fit_transform(X_test)

In [ ]:
print(len(X_train))
print(len(X_test))

323
102


# MLP HybridBat

In [ ]:
from sklearn.neural_network import MLPRegressor
from niapy.problems import Problem
from niapy.task import OptimizationType, Task
from niapy.algorithms.modified import HybridBatAlgorithm

import random

def get_hyperparameters(x):
    """Get hyperparameters for solution `x`."""
    # print(x)
    activations = ('identity', 'logistic', 'tanh', 'relu')
    solvers = ('sgd', 'adam')
    hidden_layer_sizes = (int(20 + x[0] * random.randint(100, 400)),)
    activation = activations[int(x[1] * 3)]
    solver = solvers[int(x[2])]
    alpha = random.uniform(0.00001,0.5) + x[3]
    max_iter = int(x[4] * 1000 + 1)
    
    params =  {
        'hidden_layer_sizes': hidden_layer_sizes,
        'activation': activation,
        'solver': solver,
        'alpha' : alpha,
        'max_iter' : max_iter
    }
    # print(params)
    return params


def get_regressor_HBA(x):
    """Get classifier from solution `x`."""
    params = get_hyperparameters(x)
    return MLPRegressor(**params)

In [ ]:
class MLPHyperparameterOptimization(Problem):
    def __init__(self, X_train, y_train):
        super().__init__(dimension=5, lower=0, upper=1)
        self.X_train = X_train
        self.y_train = y_train

    def _evaluate(self, x):
        model = get_regressor_HBA(x)
        scores = cross_val_score(model, self.X_train, self.y_train, cv=2, n_jobs=-1)
        return scores.mean()

In [ ]:
problem = MLPHyperparameterOptimization(X_train, y_train)

# We will be running maximization for 100 iters on `problem`
task = Task(problem, max_iters=1000, optimization_type=OptimizationType.MAXIMIZATION)

algorithm = HybridBatAlgorithm(population_size=10, seed=1234)
best_params_HBA, best_accuracy = algorithm.run(task)

print('Best parameters:', get_hyperparameters(best_params_HBA))

Best parameters: {'hidden_layer_sizes': (20,), 'activation': 'relu', 'solver': 'sgd', 'alpha': 0.6277505960652626, 'max_iter': 369}


In [ ]:
default_model = MLPRegressor()
best_model_HBA = get_regressor_HBA(best_params_HBA)

default_model.fit(X_train, y_train)
best_model_HBA.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (369) reached and the optimization hasn't conve

MLPRegressor(alpha=0.93893610607013, hidden_layer_sizes=(20,), max_iter=369,
             solver='sgd')

In [ ]:
default_score_train = default_model.score(X_train, y_train)
best_score_HBA_train = best_model_HBA.score(X_train, y_train)

print('Default model accuracy:', default_score_train)
print('Best model accuracy:', best_score_HBA_train)

Default model accuracy: 0.634616379966531
Best model accuracy: 0.8977897085788257


In [ ]:
val_scores = cross_val_score(default_model, X_val, y_val)
val_scores_best_HBA = cross_val_score(best_model_HBA, X_val, y_val)

print(-1 * val_scores.mean())
print(val_scores_best_HBA.mean())

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't conve

0.6764441070128081
0.740289698665656


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (369) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


In [ ]:
default_score_test = default_model.score(X_test, y_test)
best_score_HBA_test = best_model_HBA.score(X_test, y_test)

print('Default model accuracy:', default_score_test)
print('Best model accuracy:', best_score_HBA_test)

Default model accuracy: 0.6476909269464428
Best model accuracy: 0.8705589778760081


# MLP GWO

In [ ]:
from sklearn.neural_network import MLPRegressor
from niapy.problems import Problem
from niapy.task import OptimizationType, Task
from niapy.algorithms.basic import GreyWolfOptimizer

import random


def get_hyperparameters_GWO(x):
    """Get hyperparameters for solution `x`."""
    activations = ('identity', 'logistic', 'tanh', 'relu')
    solvers = ('sgd', 'adam')
    hidden_layer_sizes = (int(20 + x[0] * random.randint(100, 400)),)
    activation = activations[int(x[1] * 3)]
    solver = solvers[int(x[2])]
    alpha = random.uniform(0.00001,0.5) + x[3]
    max_iter = int(x[4] * 1000 + 1)
    
    params =  {
        'hidden_layer_sizes': hidden_layer_sizes,
        'activation': activation,
        'solver': solver,
        'alpha' : alpha,
        'max_iter' : max_iter
    }
    # print(params)
    return params


def get_regressor_GWO(x):
    """Get classifier from solution `x`."""
    params = get_hyperparameters_GWO(x)
    return MLPRegressor(**params)

In [ ]:
class MLPHyperparameterOptimization_GWO(Problem):
    def __init__(self, X_train, y_train):
        super().__init__(dimension=5, lower=0, upper=1)
        self.X_train = X_train
        self.y_train = y_train

    def _evaluate(self, x):
        model = get_regressor_GWO(x)
        scores = cross_val_score(model, self.X_train, self.y_train, cv=2, n_jobs=-1)
        return scores.mean()

In [ ]:
problem = MLPHyperparameterOptimization_GWO(X_train, y_train)

# We will be running maximization for 100 iters on `problem`
task = Task(problem, max_iters=1000, optimization_type=OptimizationType.MAXIMIZATION)

algorithm = GreyWolfOptimizer(population_size=10, seed=1234)
best_params_GWO, best_accuracy = algorithm.run(task)

print('Best parameters:', get_hyperparameters_GWO(best_params_GWO))

Best parameters: {'hidden_layer_sizes': (20,), 'activation': 'relu', 'solver': 'sgd', 'alpha': 0.5828015922636162, 'max_iter': 563}


In [ ]:
best_model_GWO = get_regressor_GWO(best_params_GWO)
best_model_GWO.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (563) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


MLPRegressor(alpha=0.5631959737089199, hidden_layer_sizes=(20,), max_iter=563,
             solver='sgd')

In [ ]:
best_score_GWO_train = best_model_GWO.score(X_train, y_train)

print('Default model accuracy:', default_score_train)
print('Best model accuracy:', best_score_GWO_train)

NameError: ignored

In [ ]:
val_scores = cross_val_score(default_model, X_val, y_val)
val_scores_best_GWO = cross_val_score(best_model_GWO, X_val, y_val)

print(val_scores.mean())
print(val_scores_best_GWO.mean())

In [ ]:
best_score_GWO_test = best_model_GWO.score(X_test, y_test)

print('Default model accuracy:', default_score_test)
print('Best model accuracy:', best_score_GWO_test)

# MLP PSO

In [ ]:
from sklearn.neural_network import MLPRegressor
from niapy.problems import Problem
from niapy.task import OptimizationType, Task
from niapy.algorithms.basic import ParticleSwarmOptimization

import random

def get_hyperparameters_PSO(x):
    """Get hyperparameters for solution `x`."""
    # print(x)
    activations = ('identity', 'logistic', 'tanh', 'relu')
    solvers = ('sgd', 'adam')
    hidden_layer_sizes = (int(20 + x[0] * random.randint(100, 400)),)
    activation = activations[int(x[1] * 3)]
    solver = solvers[int(x[2])]
    alpha = random.uniform(0.00001,0.5) + x[3]
    max_iter = int(x[4] * 1000 + 1)
    
    params =  {
        'hidden_layer_sizes': hidden_layer_sizes,
        'activation': activation,
        'solver': solver,
        'alpha' : alpha,
        'max_iter' : max_iter
    }
    # print(params)
    return params


def get_regressor_PSO(x):
    """Get classifier from solution `x`."""
    params = get_hyperparameters_PSO(x)
    return MLPRegressor(**params)

In [ ]:
class ABRHyperparameterOptimization_PSO(Problem):
    def __init__(self, X_train, y_train):
        super().__init__(dimension=5, lower=0, upper=1)
        self.X_train = X_train
        self.y_train = y_train

    def _evaluate(self, x):
        model = get_regressor_PSO(x)
        scores = cross_val_score(model, self.X_train, self.y_train, cv=2, n_jobs=-1)
        return scores.mean()

In [ ]:
problem = ABRHyperparameterOptimization_PSO(X_train, y_train)

# We will be running maximization for 100 iters on `problem`
task = Task(problem, max_iters=1000, optimization_type=OptimizationType.MAXIMIZATION)

algorithm = ParticleSwarmOptimization(population_size=10, seed=1234)
best_params_PSO, best_accuracy = algorithm.run(task)

print('Best parameters:', get_hyperparameters_PSO(best_params_PSO))

In [ ]:
best_model = get_regressor_PSO(best_params_PSO)
best_model.fit(X_train, y_train)

In [ ]:
best_score_PSO_train = best_model.score(X_train, y_train)

print('Default model accuracy:', default_score_train)
print('Best model accuracy:', best_score_PSO_train)

In [ ]:
val_scores = cross_val_score(default_model, X_val, y_val)
val_scores_best_PSO = cross_val_score(best_model_PSO, X_val, y_val)

print(val_scores.mean())
print(val_scores_best_PSO.mean())

In [ ]:
best_score_PSO_test = best_model.score(X_test, y_test)

print('Default model accuracy:', default_score_test)
print('Best model accuracy:', best_score_PSO_test)

# Result

In [ ]:
print('Best parameters:', get_hyperparameters(best_params_HB))
print('Best parameters:', get_hyperparameters_GWO(best_params_GWO))
print('Best parameters:', get_hyperparameters_PSO(best_params_PSO))

In [ ]:
print(best_score_HBA_train)
print(best_score_GWO_train)
print(best_score_PSO_train)

In [ ]:
print(val_scores_best_HBA.mean())
print(val_scores_best_GWO.mean())
print(val_scores_best_PSO.mean())

In [ ]:
print(best_score_HBA_test)
print(best_score_GWO_test)
print(best_score_PSO_test)